In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from transformers import Trainer, TrainingArguments

In [2]:
import numpy as np

In [3]:
def train_test_split(ratio,dataset):
    train_size = int(ratio * len(dataset))
    test_size = len(dataset) - train_size
    return random_split(dataset, [train_size, test_size])

# Swin Transformer V1

In [94]:
from transformers import AutoImageProcessor, SwinForImageClassification

image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [93]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="D:/data/kidney_dataset/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")

Resolving data files:   0%|          | 0/12446 [00:00<?, ?it/s]

In [95]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 12446
    })
})

In [96]:
dataset['train']

Dataset({
    features: ['image', 'label'],
    num_rows: 12446
})

In [97]:
dataset = dataset['train'].train_test_split(test_size=0.1)

In [128]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [117]:
def model_init(trial):
    return SwinForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

In [100]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

In [101]:
import optuna

def search_space_swinv1(trial):
    return {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-4),  # Learning rate search space
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [5, 10, 15]),  # Number of training epochs
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),  # Batch size per device during training
        "weight_decay": trial.suggest_loguniform("weight_decay", 1e-5, 1e-3),  # Weight decay
        "attention_dropout": trial.suggest_uniform("attention_dropout", 0.1, 0.3),  # Attention dropout rate
        "dropout": trial.suggest_uniform("dropout", 0.1, 0.3),  # Dropout rate
        # You can add more hyperparameters to the search space as needed
    }



In [102]:
def compute_metrics(eval_pred):
    logits,labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metrics.compute(predictions=predictions,references=labels)

In [115]:
training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        logging_dir='./logs',  # Use the trial log directory
        logging_steps=100,
        save_steps=1000,    
)

In [104]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 11201
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1245
    })
})

In [105]:
dataset['train'].set_transform(image_processor)
dataset['test'].set_transform(image_processor)

In [106]:
dataset['train']

Dataset({
    features: ['image', 'label'],
    num_rows: 11201
})

In [122]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors="tf")

In [129]:
trainer = Trainer(
    model=None,
    train_dataset =train_loader,
    eval_dataset = test_loader,
    compute_metrics=compute_metrics,
    model_init=model_init,
    args=training_args,
    # data_collator=data_collator
    
)

In [130]:
best_trail = trainer.hyperparameter_search(
    direction="maximize",
    hp_space = search_space_swinv1,
     n_trials=20,

)

[I 2024-04-07 09:03:26,350] A new study created in memory with name: no-name-5517406d-1098-485c-864e-30f1623ba7c9
C:\Users\Sid Sitaula\AppData\Local\Temp\ipykernel_1264\1920647734.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-4),  # Learning rate search space
C:\Users\Sid Sitaula\AppData\Local\Temp\ipykernel_1264\1920647734.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": trial.suggest_loguniform("weight_decay", 1e-5, 1e-3),  # Weight decay
C:\Users\Sid Sitaula\AppData\Local\Temp\ipykernel_1264\1920647734.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This f

TypeError: 'DataLoader' object is not subscriptable

In [29]:
from datasets import load_dataset
dataset = load_dataset("emotion")

D:\Anaconda\envs\data_sid\Lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [31]:
train_dataset

In [30]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})